In [1]:
import psycopg2 as ps
import pandas as pd

In [2]:
df = pd.read_csv('ssports_final.csv',index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count,categories,year
0,AjRXlSwk-E0,iddaa.com | Salernitana - Torino (0-1) - Maç Ö...,2022-04-02,2034,71,7,Serie A,2022
1,F9xbW-yWtu8,iddaa.com | Brighton - Norwich (0-0) - Maç Öze...,2022-04-02,3021,97,8,Premier League,2022
2,leUGYEfLUBY,iddaa.com | Wolves - Aston Villa (2-1) - Maç Ö...,2022-04-02,24705,489,31,Premier League,2022
3,R2f6vHRvOJ0,Andrea Belotti İle Torino 1-0 Öne Geçti! #SerieA,2022-04-02,2497,60,5,Serie A,2022
4,SKZ8pxV-KGw,iddaa.com | Manchester United - Leicester City...,2022-04-02,186284,3546,279,Premier League,2022


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 0 to 548
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       549 non-null    object
 1   video_title    549 non-null    object
 2   upload_date    549 non-null    object
 3   view_count     549 non-null    int64 
 4   like_count     549 non-null    int64 
 5   comment_count  549 non-null    int64 
 6   categories     549 non-null    object
 7   year           549 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 38.6+ KB


In [18]:
df['video_id'].duplicated().sum()

4

In [19]:
df.drop_duplicates()

,video_id,video_title,upload_date,view_count,like_count,comment_count,categories,year
0,AjRXlSwk-E0,iddaa.com | Salernitana - Torino (0-1) - Maç Ö...,2022-04-02,2034,71,7,Serie A,2022
1,F9xbW-yWtu8,iddaa.com | Brighton - Norwich (0-0) - Maç Öze...,2022-04-02,3021,97,8,Premier League,2022
2,leUGYEfLUBY,iddaa.com | Wolves - Aston Villa (2-1) - Maç Ö...,2022-04-02,24705,489,31,Premier League,2022
3,R2f6vHRvOJ0,Andrea Belotti İle Torino 1-0 Öne Geçti! #SerieA,2022-04-02,2497,60,5,Serie A,2022
4,SKZ8pxV-KGw,iddaa.com | Manchester United - Leicester City...,2022-04-02,186284,3546,279,Premier League,2022
...,...,...,...,...,...,...,...,...
544,_5a83HAcN3E,Merih Demiral Kanlar İçinde Kaldı ve Üstüne Sa...,2021-09-18,1412971,17328,971,Serie A,2021
545,kE_99E_GgzY,iddaa.com | Manchester United - Newcastle Unit...,2021-09-11,3480105,68015,4585,Premier League,2021
546,IWiiWKVWpjY,Olaylı Start | Formula 1 I 2021 Britanya GP,2021-07-18,590016,3673,614,Formula 1,2021
547,NMBV95LjgLY,Rey Mysterio ve Oğlu İntikam Peşinde! | WWE Sm...,2021-06-12,1072358,12163,1130,WWE,2021


In [20]:
df = df.drop_duplicates()

In [25]:
df.duplicated().sum()

0

In [26]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL,
                    categories TEXT NOT NULL,
                    year INTEGER NOT NULL
                    
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count, categories, year):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count,comment_count, categories, year)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count, categories, year)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, comment_count, categories, year):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s,
                categories = %s,
                year = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id, categories, year)
    curr.execute(query, vars_to_update)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['comment_count'], row['categories'], row['year'])


def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count', 'categories', 'year'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'],row['categories'],row['year'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [27]:
#Main

#database credentials
host_name = 'XXX'
dbname = 'XXX'
port = '5432'
username = 'XXX' 
password = 'XXX'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [28]:
#create table

create_table(curr)

In [29]:
#update data for existing videos

new_vid_df = update_db(curr,df)
conn.commit()

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

...

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\kadir\AppData\Local\Temp\ipykernel_17828\520374588.py:74: FutureWarning

In [30]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [32]:
#view data in db table

curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

[('AjRXlSwk-E0', 'iddaa.com | Salernitana - Torino (0-1) - Maç Özeti - Serie A 2021/22', datetime.date(2022, 4, 2), 2034, 71, 7, 'Serie A', 2022), ('F9xbW-yWtu8', 'iddaa.com | Brighton - Norwich (0-0) - Maç Özeti - Premier League 2021/22', datetime.date(2022, 4, 2), 3021, 97, 8, 'Premier League', 2022), ('leUGYEfLUBY', 'iddaa.com | Wolves - Aston Villa (2-1) - Maç Özeti - Premier League 2021/22', datetime.date(2022, 4, 2), 24705, 489, 31, 'Premier League', 2022), ('R2f6vHRvOJ0', 'Andrea Belotti İle Torino 1-0 Öne Geçti! #SerieA', datetime.date(2022, 4, 2), 2497, 60, 5, 'Serie A', 2022), ('SKZ8pxV-KGw', 'iddaa.com | Manchester United - Leicester City (1-1) - Maç Özeti - Premier League 2021/22', datetime.date(2022, 4, 2), 186284, 3546, 279, 'Premier League', 2022), ('dBzterJAn3Q', 'iddaa.com | Lazio - Sassuolo (2-1) - Maç Özeti - Serie A 2021/22', datetime.date(2022, 4, 2), 13085, 263, 30, 'Serie A', 2022), ('klnWbxooZf4', 'Lazio, Sergej Milinkovic-Savic ile farkı ikiye çıkarıyor!  #Seri

In [33]:
new_vid_df

,video_id,video_title,upload_date,view_count,like_count,comment_count,categories,year
0,AjRXlSwk-E0,iddaa.com | Salernitana - Torino (0-1) - Maç Ö...,2022-04-02,2034,71,7,Serie A,2022
1,F9xbW-yWtu8,iddaa.com | Brighton - Norwich (0-0) - Maç Öze...,2022-04-02,3021,97,8,Premier League,2022
2,leUGYEfLUBY,iddaa.com | Wolves - Aston Villa (2-1) - Maç Ö...,2022-04-02,24705,489,31,Premier League,2022
3,R2f6vHRvOJ0,Andrea Belotti İle Torino 1-0 Öne Geçti! #SerieA,2022-04-02,2497,60,5,Serie A,2022
4,SKZ8pxV-KGw,iddaa.com | Manchester United - Leicester City...,2022-04-02,186284,3546,279,Premier League,2022
...,...,...,...,...,...,...,...,...
544,_5a83HAcN3E,Merih Demiral Kanlar İçinde Kaldı ve Üstüne Sa...,2021-09-18,1412971,17328,971,Serie A,2021
545,kE_99E_GgzY,iddaa.com | Manchester United - Newcastle Unit...,2021-09-11,3480105,68015,4585,Premier League,2021
546,IWiiWKVWpjY,Olaylı Start | Formula 1 I 2021 Britanya GP,2021-07-18,590016,3673,614,Formula 1,2021
547,NMBV95LjgLY,Rey Mysterio ve Oğlu İntikam Peşinde! | WWE Sm...,2021-06-12,1072358,12163,1130,WWE,2021
